In [113]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [125]:
N_EPOCHS = 100
N_TRIALS = 3
BATCH_SIZE = 256

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [131]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

In [132]:
x_train = np.clip(x_train + np.random.normal(0, 40, x_train.shape), 0, 255)
x_val = np.clip(x_val + np.random.normal(0, 40, x_val.shape), 0, 255)
x_test = np.clip(x_test + np.random.normal(0, 40, x_test.shape), 0, 255)

In [133]:
assessment_model = ClassifierModel(output_shape=10, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
assessment_model.fit(x_train, y_train, epochs=60, batch_size=BATCH_SIZE)

Epoch 1/60
28/28 [==============================] - 1s 30ms/step - loss: 2.5757 - accuracy: 0.1180
Epoch 2/60
28/28 [==============================] - 1s 28ms/step - loss: 2.3181 - accuracy: 0.1704
Epoch 3/60
28/28 [==============================] - 1s 28ms/step - loss: 2.1645 - accuracy: 0.2104
Epoch 4/60
28/28 [==============================] - 1s 29ms/step - loss: 2.0815 - accuracy: 0.2423
Epoch 5/60
28/28 [==============================] - 1s 28ms/step - loss: 2.0188 - accuracy: 0.2609
Epoch 6/60
28/28 [==============================] - 1s 28ms/step - loss: 1.9558 - accuracy: 0.2744
Epoch 7/60
28/28 [==============================] - 1s 31ms/step - loss: 1.8975 - accuracy: 0.3097
Epoch 8/60
28/28 [==============================] - 1s 28ms/step - loss: 1.8552 - accuracy: 0.3091
Epoch 9/60
28/28 [==============================] - 1s 28ms/step - loss: 1.8165 - accuracy: 0.3320
Epoch 10/60
28/28 [==============================] - 1s 37ms/step - loss: 1.7789 - accuracy: 0.3429
Epoch 11/

In [134]:
y_pred_assessment = assessment_model.predict(x_train, verbose=0)

losses_assessment = utils.calculate_loss_per_sample(y_train, y_pred_assessment, loss)
order_assessment = np.argsort(losses_assessment)

In [147]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [148]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [149]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(2 * (i + 1)/ (N_EPOCHS / 2)) * train_size)
            samples_ids = order_assessment[:n_samples]
            
            model.fit(
                x_train[samples_ids], y_train[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE,
                verbose=1, callbacks=[early_stopping]
            )
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
        
        model_scores.append(accuracy)
    
    results[test_model] = model_scores

test_model_1


  0%|          | 0/3 [00:00<?, ?it/s]

6/6 [==============================] - 0s 15ms/step - loss: 1.3150 - accuracy: 0.5247


 33%|███▎      | 1/3 [02:06<04:13, 126.51s/it]

6/6 [==============================] - 0s 13ms/step - loss: 1.5527 - accuracy: 0.4620


 67%|██████▋   | 2/3 [04:16<02:08, 128.65s/it]

6/6 [==============================] - 0s 11ms/step - loss: 1.4489 - accuracy: 0.4847


100%|██████████| 3/3 [06:28<00:00, 129.43s/it]


In [150]:
results

{'test_model_1': [0.5246666669845581, 0.4620000123977661, 0.4846666753292084]}

In [ ]:
{'test_model_1': [0.5386666655540466, 0.5506666898727417, 0.5633333325386047]}

In [107]:
(np.tanh(2 * (200 + 1)/ N_EPOCHS))

0.9647273193205547

In [112]:
(np.tanh(2 * (50 + 1)/ (N_EPOCHS / 2)))

0.7698665359089003